In [ ]:
!pip install transformers
!pip install torch
!pip install sentence-transformers
!pip install faiss-cpu
!pip install flask
!pip install pyngrok
!pip install -U bitsandbytes
!pip install pyyaml
!pip install flask-ngrok

In [2]:

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
import faiss
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import BitsAndBytesConfig
import os
import re #used for the reductor
import yaml
import unicodedata
import hashlib
from collections import OrderedDict
from typing import List, Dict, Tuple
from flask_ngrok import run_with_ngrok
import requests
import time
from threading import Thread
import traceback


In [ ]:
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "ShopLite User Registration Process",
        "content": """To create a ShopLite account, users navigate to the registration page and provide a valid email address along with a strong password. A verification email is sent to confirm the account. Users can optionally add personal information such as name, phone number, and shipping address. Once the account is activated, users gain access to the dashboard where they can manage personal information, payment methods, and view past orders. Passwords are securely hashed, and ShopLite encourages enabling two-factor authentication for enhanced security. Forgotten passwords can be reset via a secure link sent to the registered email.
From a technical standpoint, the system validates inputs to prevent duplicate accounts and ensures proper email formatting. Backend processes include logging account creation events, updating the user database, and triggering welcome emails. API endpoints exist for retrieving and updating account data programmatically, enabling integration with mobile apps or external services. Error handling ensures that failed verifications, weak passwords, or invalid input are promptly communicated to the user. Progressive form validation improves the UX by giving immediate feedback on missing or incorrect fields. The registration process also considers internationalization by supporting multiple languages and local phone number formats. These combined steps aim to ensure security, usability, and reliability, reducing friction for new users while safeguarding sensitive information.
Users who no longer wish to use ShopLite can request account deactivation or permanent deletion through the settings page or customer support. Deactivation temporarily suspends access while preserving data, whereas deletion permanently removes personal information and order history in compliance with privacy regulations. Notifications and confirmations are provided to ensure users are fully informed before completing these actions."""
    },

    {
        "id": "doc2",
        "title": "ShopLite Shopping Cart Features",
        "content":"""ShopLite’s shopping cart allows Lebanese users to add products from multiple categories, modify quantities, and remove items. The cart calculates subtotals, applicable taxes, and shipping fees based on Lebanese zones. Users can save items for later or create wish lists. Checkout validates stock availability, shipping addresses, and payment methods before confirming orders. One-click checkout and saved local payment options streamline repeat purchases.
Backend systems check inventory to prevent overselling, manage sessions, and log actions. Invalid promo codes or insufficient stock trigger immediate feedback. Supported payment methods include Bank Audi, BLOM Bank, Fransabank cards, bank transfers, and OwlPay. The mobile interface supports touch interactions, responsive layouts, and dynamic updates. Caching frequent cart data improves performance. APIs allow external services to query or update carts programmatically. Edge cases include expired coupons, partial availability, and split shipments. Overall, the cart is designed to enhance user experience, reduce abandonment, and ensure transactional accuracy. """
    },

    {
        "id": "doc3",
        "title": "Product Search and Filtering",
        "content":"""ShopLite provides a comprehensive product search system designed to help users quickly find the items they want. Users can enter keywords, filter by categories, price in LBP or USD, color, size, or sort by relevance, rating, or popularity. AI-powered typeahead suggests completions and relevant categories as users type, improving search efficiency. Search results include seller ratings, product reviews, and stock availability indicators to help users make informed decisions.
Technically, the system uses inverted indexes and semantic embeddings for efficient retrieval. FAISS-based vector search enables quick similarity matching for natural language queries. Caching popular search results reduces latency for repeated queries. Advanced filters allow combining multiple criteria, and backend validation ensures filter selections match available attributes. APIs provide developers with endpoints to query products using various parameters programmatically. Search relevance is continually optimized using user behavior analytics, click-through rates, and AI ranking algorithms. Edge cases include handling ambiguous terms, misspellings, and returning recommendations when no exact matches are found. Overall, the search system balances speed, accuracy, and user experience, guiding buyers from query to checkout efficiently. """
    },

    {
        "id": "doc4",
        "title": "Payment Methods and Security ",
        "content":""" ShopLite provides a variety of payment options to make transactions convenient and secure for users in Lebanon. Customers can pay using credit and debit cards from Bank Audi, BLOM Bank, and Fransabank (Visa and MasterCard), as well as direct bank transfers to these banks. Selected local mobile wallets, such as OwlPay , are also supported for users who prefer mobile payment methods, and alternative digital payment methods like Wish Money. All transactions are secured using SSL/TLS encryption, and sensitive data, such as card numbers or bank account information, is tokenized to prevent exposure or theft.
To protect against fraud, ShopLite implements multiple security checks, including velocity checks to detect rapid repeated transactions, IP verification to identify unusual access locations, device fingerprinting, and AI-powered anomaly detection to flag suspicious activity. Both frontend and backend validation are in place to prevent malformed or fraudulent transactions from being processed. If a payment fails due to a declined card, insufficient funds, or expired token, the system provides immediate feedback to the user so they can correct the issue.
For refunds, ShopLite processes returns to the original payment method once approval is confirmed, ensuring a smooth user experience. Developers and sellers can use ShopLite APIs to programmatically initiate payments, confirm transactions, or issue refunds as needed, integrating payment flows into external systems or applications.
All payment actions are logged for auditing, monitoring, and compliance purposes, helping ensure accountability and traceability. By combining locally relevant payment options, robust security measures, automated processing, and API accessibility, ShopLite ensures that users can complete transactions quickly, safely, and reliably, building trust and convenience for buyers and sellers alike. """
    },

    {
        "id": "doc5",
        "title": "Order Tracking and Delivery  ",
        "content":""" ShopLite provides users with real-time order tracking through email notifications, mobile alerts, and updates on the dashboard. Every order is assigned a unique tracking number, which is synchronized with the selected local courier to ensure accurate monitoring throughout the delivery process. Estimated delivery times are calculated based on stock location, courier performance, and the chosen shipping method. Users can request rescheduling, provide additional instructions, or contact support for assistance at any stage.
Shipping is handled by trusted local couriers, including Aramex Lebanon, TNT Lebanon, and Fetchr Lebanon, with each carrier integrated into the backend system. Inventory levels are synchronized with dispatch events to prevent overselling, while automated notifications alert customer service teams and users in the event of delays or failed deliveries. APIs allow programmatic retrieval of tracking information, updates on shipment status, and management of exceptions such as returns or cancellations.
Analytics track delivery times, exceptions, and performance metrics to identify areas for improvement and optimize logistics efficiency. Edge cases, such as split shipments, partial availability of products, or address corrections, are handled automatically to minimize disruptions. Inter-city deliveries are supported, with recalculated estimates if stock locations change during processing.
By combining real-time tracking, reliable courier partnerships, automated inventory synchronization, and monitoring analytics, ShopLite ensures a transparent, efficient, and dependable delivery experience. Customers receive timely updates and complete visibility of their order progress, contributing to higher satisfaction and trust in the platform."""
    },

    {
        "id": "doc6",
        "title": " Return and Refund Policies ",
        "content":"""ShopLite allows users to request returns within 14–30 days depending on the product category. Users submit return requests via the dashboard, specifying reasons and selecting return methods, such as courier pickup or drop-off at designated locations. Returns are approved after inspection for condition and eligibility. Refunds are processed to the original payment method within 3–7 business days after approval. Certain products, including perishables or digital items, may be exempt.
Backend systems maintain detailed logs for each return, tracking submission, inspection, and refund execution. For sellers, the platform enforces rules for return eligibility, commission adjustments, and inventory updates. Automated systems handle exceptions, escalating disputes to customer support when necessary. Users receive notifications at every stage of the return process. API endpoints allow developers to initiate, track, and update returns programmatically. By combining clear policy communication, automated backend processing, and notifications, ShopLite ensures a smooth and trustworthy return experience for customers and sellers alike."""
    },

    {
        "id": "doc7",
        "title": " Product Reviews and Ratings ",
        "content":""" ShopLite allows users to submit ratings from 1–5 stars along with written reviews for products they have purchased, helping other buyers make informed decisions. To maintain authenticity, purchase verification ensures that only users who have actually bought the product can leave a review. Reviews influence search rankings, product visibility, and recommendation systems, meaning that higher-rated products are more prominently displayed, while sellers with consistent positive feedback can build trust and credibility.
To maintain quality and safety, all reviews are subject to AI-based moderation and manual review. This filters out offensive, spammy, or inappropriate content, ensuring that customers see only helpful and relevant feedback. Aggregated ratings and summaries are displayed on product pages, seller dashboards, and analytics tools, giving both buyers and sellers clear insights into product performance. Notifications alert sellers when new reviews are posted, enabling them to respond promptly to customer feedback, address issues, and improve service quality.
ShopLite provides APIs that allow developers to access reviews programmatically for analytics, reporting, or integration with external dashboards and recommendation engines. Technical considerations include caching frequently requested queries to reduce latency, indexing reviews for semantic search to improve relevance, and ensuring privacy compliance to protect user data.
By combining verified reviews, moderation, notifications, analytics, and API access, ShopLite creates a trustworthy and transparent review system. This helps buyers make informed purchasing decisions, motivates sellers to maintain high-quality products and services, and supports data-driven improvements across the platform. Overall, the reviews and ratings system strengthens both the user experience and seller accountability, contributing to a more reliable and engaging marketplace. """
    },

    {
        "id": "doc8",
        "title": " Seller Account Setup and Management ",
        "content":"""Sellers register through a dedicated portal, providing business verification documents, and tax information. Once approved, sellers can create product listings, set pricing, manage inventory, and track orders. Dashboards display metrics such as sales volume, returns, and average ratings.
Backend systems enforce access control, validate product data, and synchronize stock availability across multiple sales channels. APIs allow sellers to automate inventory updates, order retrieval, and promotional campaigns. Security measures include two-factor authentication via SMS, session monitoring, and activity logs. Sellers receive notifications for new orders, disputes, and policy updates. The system also supports local financial requirements, including VAT calculations and local bank settlement procedures. By providing transparent metrics, automation, and compliance with local regulations, ShopLite empowers sellers to manage their stores efficiently. """
    },

    {
        "id": "doc9",
        "title": "Inventory Management for Sellers",
        "content":""" ShopLite enables sellers to manage product availability, SKUs, and stock quantities through dashboards or APIs. Automated low-stock alerts notify sellers to replenish items. Bulk updates are supported via API, minimizing manual work. Inventory synchronization ensures buyers see accurate stock levels in real time.
Backend systems validate changes, enforce minimum thresholds, and integrate with order fulfillment workflows. Historical data and analytics allow sellers to forecast demand and optimize turnover. All inventory changes are logged for auditing and reporting. Edge cases, like partial availability or overselling prevention, are handled automatically. By combining real-time updates, analytics, and automated alerts, ShopLite maintains inventory accuracy and seller efficiency. """
    },

    {
        "id": "doc10",
        "title": "Commission and Fee Structure",
        "content":""" ShopLite charges commissions on sales that vary depending on the product category and type, ensuring that fees reflect the nature of the transaction. In addition to standard commissions, additional fees may apply for promotions, priority listings, advertising campaigns, or special services that enhance product visibility or seller benefits. Sellers can view a detailed breakdown of all fees and commissions in their dashboards, which are updated in real time, or access the information programmatically through ShopLite APIs. This transparency allows sellers to track costs and understand how fees are applied to each transaction.
Fees are calculated automatically when an order is completed and are adjusted in cases of returns, cancellations, or disputes, ensuring that sellers are charged accurately based on actual sales. The backend accounting system reconciles all payments, deducts commissions, and generates comprehensive financial reports for sellers. Automated notifications alert sellers whenever fee structures are updated or when adjustments occur, keeping them informed and reducing confusion.
ShopLite APIs allow sellers to integrate financial data with external accounting software, enabling automation of bookkeeping, reconciliation, and reporting tasks. The system is designed to comply fully with local Lebanese tax regulations, ensuring accurate reporting and legal compliance. Historical records are maintained for auditing purposes and for analyzing trends in sales, fees, and returns.
By combining automated calculations, real-time notifications, API access, and compliance measures, ShopLite provides sellers with clear and reliable financial visibility, helping them manage their operations efficiently, plan for profitability, and make informed business decisions with confidence """
    },

    {
        "id": "doc11",
        "title": "Customer Support Procedures",
        "content":""" ShopLite provides multi-channel support through chat, email, and phone, focusing on assisting users with order tracking and related inquiries. An AI support assistant handles questions about order status, delivery updates, shipment tracking, and related procedures, providing fast and accurate responses by retrieving information from the ShopLite system. For queries that fall outside its scope or are too complex—such as disputes, payment issues, or account problems—the AI escalates the request to human agents, who follow defined protocols to ensure proper resolution.
The support system tracks key metrics, including first response time, resolution rate, and customer satisfaction, allowing ShopLite to monitor and optimize performance. Support dashboards provide agents with visibility into all open tickets, escalations, and SLA adherence, ensuring timely handling of user requests. Technical documentation includes API endpoints for ticket creation, updates, and notifications, enabling integration with the AI assistant and other systems for streamlined operations.By combining focused AI assistance, human oversight for exceptions, monitoring dashboards, and secure data handling, ShopLite provides a reliable support experience specifically for order-related queries. Users can quickly check the status of their orders, receive updates, and resolve delivery concerns efficiently, maintaining confidence and satisfaction in the platform. """
    },

    {
        "id": "doc12",
        "title": "Mobile App Features",
        "content":"""The ShopLite mobile app offers users functionality that closely mirrors the website, providing a seamless experience across devices. Users can manage their accounts, browse and search products, add items to their cart, complete checkout, and receive notifications directly within the app. Additional features include the ability to scan QR codes, save favorite products for later, apply promotional codes at checkout, and track orders in real time. Push notifications keep users informed about discounts, promotions, order updates, and important announcements, enhancing engagement and timely decision-making.
The app’s user experience (UX) is optimized for mobile, supporting intuitive touch gestures, fast loading times, and offline access, allowing users to browse products even with limited connectivity. Prices are displayed in Lebanese pounds (LBP) to ensure local relevance. APIs synchronize data with the backend continuously, maintaining real-time consistency between the mobile app, website, and ShopLite databases.
Security is a priority, the mobile app includes device authentication, biometric login (fingerprint or face ID), and secure storage of sensitive data, ensuring user information and payment details are protected. Analytics within the app track user engagement, shopping behavior, and overall app performance, providing insights for future improvements.
By combining responsive design, offline capabilities, seamless backend synchronization, secure authentication, and local contextualization, ShopLite ensures a smooth, reliable, and user-friendly mobile shopping experience. Users enjoy the convenience of managing their shopping, payments, and account details anytime, while the platform maintains security, performance, and consistency across all channels, creating a fully integrated and satisfying mobile experience."""
    },

    {
        "id": "doc13",
        "title": " API Documentation for Developers",
        "content":"""ShopLite provides a set of RESTful APIs that allow developers to access and manage products, orders, inventory, and payments programmatically. These APIs are designed to make integrations with external systems, mobile apps, or dashboards easy and reliable. To prevent abuse, ShopLite requires API keys for authentication and enforces rate limits, ensuring fair usage and protecting system performance. The main endpoints include /products for retrieving product details, /orders/{id} for accessing specific order information, /inventory/update for managing stock levels, and /payments/process for initiating and confirming payments. All responses are returned in JSON format, with clear error codes and messages to help developers identify issues quickly.
Best practices for using the APIs include input validation to ensure correct parameters, retry logic to handle temporary failures, and monitoring latency to maintain performance. ShopLite also provides detailed technical documentation that includes example calls, parameter descriptions, authentication steps, error codes, and integration guidelines. Developers can rely on this documentation to implement API calls correctly and efficiently.
By combining secure authentication, rate limiting, clear error handling, and thorough documentation, ShopLite’s APIs enable developers to build robust, reliable, and scalable integrations. This ensures that external systems can interact seamlessly with the ShopLite platform, supporting smooth operations for sellers, buyers, and third-party applications."""
    },

    {
        "id": "doc14",
        "title": "Security and Privacy Policies",
        "content":""" ShopLite prioritizes data security and privacy across all operations. All user and seller data is encrypted in transit and at rest using industry-standard protocols. Role-based access controls ensure that only authorized personnel can access sensitive information, while detailed audit logs record all critical system activity for accountability and troubleshooting. Privacy settings allow users to control consent, data sharing, and visibility of personal information.
The platform’s policies comply with GDPR, CCPA, and relevant local regulations, providing a framework for lawful and responsible data handling. Regular vulnerability scans, penetration testing, and continuous monitoring help identify and address potential security threats before they impact the system. Personal data used for analytics or reporting is anonymized to protect individual privacy. All API access requires secure authentication and activity logging to prevent unauthorized use.
ShopLite maintains a comprehensive incident response plan, including breach detection, notification, mitigation, and recovery procedures. Security extends to mobile and web applications, ensuring that sensitive information such as account credentials, payment data, and order history is stored and transmitted securely. By combining encryption, access control, monitoring, and compliance, ShopLite protects both operational and user data. These measures provide confidence that privacy and security are consistently enforced across the platform while enabling smooth and trustworthy interactions for all users. """
    },

    {
        "id": "doc15",
        "title": "Promotional Codes and Discounts",
        "content":"""ShopLite allows sellers to create and manage promotional codes with flexible conditions, including expiration dates, minimum purchase amounts, applicable product categories, or specific seller campaigns. Discounts can be percentage-based, fixed-amount, or buy-one-get-one (BOGO) offers. Customers apply promotional codes during checkout, where the system automatically validates eligibility, calculates the correct discount, and updates the order total in real time.
The backend integrates promotional logic with inventory and cart systems to ensure accurate application and prevent misuse. Analytics track code usage, redemption rates, and campaign effectiveness, providing sellers with insights on promotional performance and customer engagement. Notifications alert customers to active promotions, upcoming expirations, or special campaigns, helping increase participation and conversions.
Developers can use ShopLite APIs to create, update, or retrieve promotional codes programmatically, enabling integration with external marketing tools, automated campaigns, or loyalty programs. Edge cases such as expired codes, invalid products, or multiple overlapping promotions are handled gracefully to prevent errors at checkout. Detailed logging ensures auditing and allows troubleshooting of promotional campaigns.
By combining flexible promotional rules, real-time validation, analytics, and integration, ShopLite empowers sellers to increase sales, improve engagement, and deliver a seamless promotional experience for customers. The system ensures transparency, accuracy, and effectiveness of every promotional initiative across the platform. """
    }
]

# i'll apply chunking to the knowledge base so that when embedding the data it won't be blurry and it makes retrieval easier and more precise
# note : I applied chunking because initial retrievals often returned irrelevant results
def chunk_text(text: str, chunk_size: int = 150, overlap: int = 30) -> List[str]:
    words = re.split(r"\s+", text)
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if chunk:
            chunks.append(chunk)
    return chunks

def expand_kb_with_chunks(KNOWLEDGE_BASE: List[Dict], chunk_size=150, overlap=30):
    expanded_kb = []
    for doc in KNOWLEDGE_BASE:
        chunks = chunk_text(doc["content"], chunk_size, overlap)
        for idx, chunk in enumerate(chunks):
            expanded_kb.append({
                "id": f"{doc['id']}_chunk{idx}",
                "title": doc["title"],
                "content": chunk,
                "parent_doc": doc["id"]
            })
    return expanded_kb


EXPANDED_KB = expand_kb_with_chunks(KNOWLEDGE_BASE)
# print(f"Original docs: {len(KNOWLEDGE_BASE)}, Expanded chunks: {len(EXPANDED_KB)}")

In [ ]:
PROMPTS_YAML="""
# This file is used to configure the assistant's behavior and personality
# It is written in YAML format
# Each section defines a different scenario for the assistant's behavior, such as retrieval, refusal, or clarification, rather than combining them into a single section.

version: "1.0"
created: "2025-09-27"
author: "Charlie Sukkar"
description: "Configuration file for the AI assistant's behavior and personality."

Guidelines: &base
  Audiences:
    - "ShopLite customers and sellers who need accurate guidance based solely on the official ShopLite documentation."
    - "Ensure that your responses are culturally appropriate and relevant to the Lebanese market."
  Tone:
    - "Maintain a friendly, professional, and empathetic tone."
    - "Be concise and to the point, avoiding unnecessary jargon."
    - "Use clear and simple language that is easy to understand."
  Instructions:
    - "Answer in one concise paragraph directed to the user."
    - "Do not list items, do not number points."
    - "Only include essential information relevant to the query."
    - "Mention document titles in parentheses if needed."
    - "Keep the answer short and clear (ideally 3-6 sentences)."
    - "do not invent information, Only answer using the context provided"
  safety: |
    No PII should be processed or included in responses.


base-retrieval-prompt:
  role: "You are a helpful Shoplite customer service assistant."
  goal: |
    Assist customers by providing accurate, concise, and helpful answers using only the provided Shoplite documentation.
  context:
    - "Your only source of information is the official Shoplite knowledge base."
    - "Always cite specific document titles and sections from the Shoplite document snippets when possible."
    - "Inform users if documentation or policies have recently changed"
    - "If the user asks for all ShopLite policies, provide a direct link to the official policy document file: .docs/prompting/knowledge-base.md"
  Guidelines: *base
  Response-format: |
    Answer: [Provide a concise, user-friendly summary based on context, not a long list]
    Sources: [List document titles referenced]
    Confidence: "High"
    ###end

multi-doc-retrieval-prompt:
  role: "You are a knowledgeable ShopLite customer service assistant capable of synthesizing information from multiple documents to provide comprehensive answers."
  goal: |
    you have to provide accurate and complete responses by combining information from multiple documents
    Ensure that all included details are supported by the knowledge base and Clearly cite the sources
  context:
    - "cite each document used in your response "
    - "ensure the response is relevant to the user query"
    - "If the answer requires interpretation across documents, summarize and avoid assumptions"
  Guidelines: *base
  Response-format: |
    Answer: [Provide a concise, user-friendly summary based on context, not a long list]
    Sources: [List document titles referenced]
    Confidence: "Medium"
    ###end


refusal-prompt:
  role: "you are a reliable and responsible Shoplite customer service assistant "
  goal: |
    politely refuse to answer questions that cannot be supported by the official ShopLite knowledge base
    and ensure to tell the user that only the data in the documents can be provided
  context:
    - "if the requested data is not in the knowledge base do not guess or provide external advice"
    - "tell the user why you can't provide an answer"
  Guidelines: *base
  Response-format: |
    Answer: "I'm sorry, I don't have information about that based on the official ShopLite documentation."
    Source: "none"
    Confidence: "Low"
    ###end

clarification-prompt:
  role: " you are a customer service assistant, skilled at identifying when questions need clarification"
  goal: "  Politely request clarification when a user's question is ambiguous or lacks enough detail to provide a reliable answer."
  context:
    - "Do not provide answers if the query is unclear or could refer to multiple topics."
    - "Ask targeted questions to gather the missing information"
  Guidelines: *base
  Response-format: |
    Answer: "Could you please clarify [specific part of the query] so I can provide an accurate response?"
    Source: "none"
    Confidence: "Medium"
    ###end
"""
PROMPTS = yaml.safe_load(PROMPTS_YAML)
#print(PROMPTS["base-retrieval-prompt"]["role"])

In [ ]:
from huggingface_hub import login


hf_token = input("Enter your Hugging Face token: ")
login(token=hf_token)

In [ ]:

#  for week 5 I changed the model , i used  Qwen2.5-1.5B-Instruct bcz it loads faster and uses less memory on Colab's free tier, so the assistant stays online longer without crashing between usage.

# model_name = "meta-llama/Llama-3.1-8B"


# tokenizer = AutoTokenizer.from_pretrained(model_name)

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     device_map="auto"
# )


# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Test the model
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

print("✅ Model loaded successfully!")

In [ ]:
# in this cell we will create 2 functions for normalization and reduction of personal info(PII)
def normalize_text(s: str) -> str:
    s = s.strip()
    s = s.lower()
    s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    s = re.sub(r'\s+', ' ', s)
    return s

PII_PATTERNS = [
    r'\b\d{12,19}\b',                  # credit/debit cards
    r'\b\d{7,8}\b',                    # Lebanese national ID
    r'(\+961\d{7,8})|\b0\d{7,8}\b',    # Lebanese phone numbers
    r'[\w\.-]+@[\w\.-]+\.\w+',         # emails
    r'\bLB\d{20}\b',                   # Lebanese IBAN
    r'\b[0-9]{6,}\b',                  # generic long numbers (fallback)
]

def redact_pii(text: str) -> str:
    t = text
    for p in PII_PATTERNS:
        t = re.sub(p, '[REDACTED]', t)
    return t



CACHE = OrderedDict()
CACHE_MAX = 1000

def cache_get(key):
    val = CACHE.get(key)
    if val:
        CACHE.move_to_end(key)
    return val

def cache_set(key, value):
    CACHE[key] = value
    CACHE.move_to_end(key)
    if len(CACHE) > CACHE_MAX:
        CACHE.popitem(last=False)


def emb_to_key(embedding: np.ndarray) -> str:
    emb_bytes = embedding.tobytes()
    key = hashlib.sha1(emb_bytes).hexdigest()
    return key

In [ ]:

SHOPLITE_KEYWORDS = {
    'core_business': [
        'shoplite', 'shop lite', 'shopping', 'ecommerce', 'e-commerce', 'marketplace',
        'online store', 'retail', 'buy', 'sell', 'purchase', 'order', 'cart', 'checkout'
    ],
    'features': [
        'payment', 'shipping', 'delivery', 'tracking', 'account', 'registration', 'login',
        'product', 'search', 'filter', 'category', 'inventory', 'stock', 'price',
        'review', 'rating', 'seller', 'customer', 'support', 'refund', 'return',
        'commission', 'fee', 'api', 'mobile app', 'notification', 'promotion', 'discount',
        'courier', 'couriers', 'deliveries'
    ],
    'lebanese_context': [
        'lebanon', 'lebanese', 'beirut', 'lbp', 'usd', 'bank audi', 'blom bank',
        'fransabank', 'owlpay', 'wish money', 'aramex lebanon', 'tnt lebanon',
        'fetchr lebanon', 'vat', 'aramex', 'tnt', 'fetchr'
    ]
}


OUT_OF_SCOPE_INDICATORS = [
    'what is', 'who is', 'when was', 'where is', 'history of', 'definition of',
    'amazon', 'ebay', 'alibaba', 'facebook', 'google', 'apple', 'microsoft',
    'machine learning', 'artificial intelligence', 'blockchain', 'cryptocurrency',
    'programming', 'coding', 'python', 'javascript', 'database design',
    'should i', 'what do you think', 'recommend me', 'suggest me',
    'weather', 'news', 'politics', 'sports', 'entertainment', 'celebrity',
    'medical', 'health', 'legal advice', 'investment', 'stock market'
]



def create_embeddings(kb):

    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    texts = [doc['content'] for doc in kb]
    embeddings = embedder.encode(texts, normalize_embeddings=True).astype('float32')

    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)

    return index, embedder


def retrieve_docs(query, kb, index, embedder, top_k=3, similarity_threshold=0.3):

    query_norm = normalize_text(query)
    query_redacted = redact_pii(query_norm)
    query_lower=query_redacted.lower()

    q_emb = embedder.encode(query_redacted, normalize_embeddings=True).astype('float32')

    cache_key = emb_to_key(q_emb)
    cached = cache_get(cache_key)
    if cached:
        return cached

    q_emb_exp = np.expand_dims(q_emb, axis=0)
    distances, indices = index.search(q_emb_exp, top_k)

    results = []
    for idx, sim in zip(indices[0], distances[0]):
        if sim >= similarity_threshold:
            results.append(kb[idx])

    if not results:
        # If no semantic matches, try keyword matching
        query_words = set(query_lower.split())
        for i, doc in enumerate(kb):
            doc_words = set(doc['content'].lower().split())
            overlap = len(query_words.intersection(doc_words))
            if overlap >= 2:
                results.append(doc)
                if len(results) >= top_k:
                    break

    cache_set(cache_key, results)
    return results

## note to the intructor: I added extra functions and emphasized the prompt structure to ensure the model would read it properly because without them it often failed to follow the format. Although I explored other open-source models like OpenChat and Zephyr, they didn’t follow the prompt reliably. I ended up sticking with LLaMA, because it handled the format best.

def is_shoplite_related(query: str) -> Tuple[bool, float]:
    query_lower = query.lower().strip()

    if any(keyword in query_lower for keyword in SHOPLITE_KEYWORDS['core_business']):
        return True, 0.9

    clear_out_of_scope = ['weather today', 'president of', 'cook pasta', 'machine learning tutorial']
    if any(phrase in query_lower for phrase in clear_out_of_scope):
        return False, 0.1

    delivery_terms = ['courier', 'delivery', 'shipping', 'aramex', 'tnt', 'fetchr']
    if any(term in query_lower for term in delivery_terms):
        return True, 0.8

    total_keywords = sum(len(keywords) for keywords in SHOPLITE_KEYWORDS.values())
    matched_keywords = 0

    for category, keywords in SHOPLITE_KEYWORDS.items():
        for keyword in keywords:
            if keyword in query_lower:
                matched_keywords += 1

    confidence = matched_keywords / total_keywords if total_keywords > 0 else 0
    threshold = 0.01

    return confidence >= threshold, confidence


def detect_question_type(query: str) -> str:
    query_lower = query.lower().strip()

    greetings = ['hello', 'hi', 'hey', 'good morning', 'good afternoon', 'good evening']
    if any(greeting in query_lower for greeting in greetings):
        return 'greeting'

    assistant_questions = ['who are you', 'what are you', 'what can you do', 'help me']
    if any(q in query_lower for q in assistant_questions):
        return 'assistant_info'


    general_patterns = [
        r'^(what is the weather|who is the president|how to cook)',
        r'(tell me about world news|explain quantum physics)',
        r'(definition of artificial intelligence)'
    ]
    if any(re.search(pattern, query_lower) for pattern in general_patterns):
        return 'general_knowledge'

    return 'unknown'


def setup_tokenizer_padding(tokenizer):

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
        print("Set pad_token to eos_token")
    return tokenizer

def setup_model_and_tokenizer():

    global tokenizer

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
        print("Set pad_token to eos_token")

    print(f"Tokenizer pad_token: {tokenizer.pad_token}")
    print(f"Tokenizer pad_token_id: {tokenizer.pad_token_id}")

    return tokenizer


def build_prompt_from_yaml(query, retrieved_docs, prompt_type="base-retrieval-prompt"):

    prompt_config = PROMPTS.get(prompt_type, PROMPTS["base-retrieval-prompt"])


    if retrieved_docs:
        context_text = "\n\n".join([f"Document: {doc['title']}\nContent: {doc['content']}" for doc in retrieved_docs])
    else:
        context_text = "No relevant documents found in the ShopLite knowledge base."


    role = prompt_config.get('role', 'You are a helpful ShopLite assistant.')
    goal = prompt_config.get('goal', 'Assist with ShopLite-related questions.')
    context_rules = prompt_config.get('context', [])
    guidelines = prompt_config.get('Guidelines', {})
    response_format = prompt_config.get('Response-format', 'Provide a helpful response.')


    prompt_parts = [
        "CRITICAL INSTRUCTIONS:",
        "- You are a ShopLite customer service assistant",
        "- Extract SPECIFIC details from the context (names, numbers, procedures)",
        "- If asked about couriers, list the specific courier names mentioned",
        "- If asked about timeframes, give the exact days/periods stated",
        "- If asked about procedures, describe the specific steps mentioned",
        "- DO NOT use general knowledge - only use the provided context",
        "- Quote exact information from the documents",
        "",
        f"Role: {role}",
        f"Goal: {goal}",
        "",
        "Context Rules:",
    ]

    if isinstance(context_rules, list):
        for rule in context_rules:
            prompt_parts.append(f"- {rule}")

    prompt_parts.extend([
        "",
        "Guidelines:",
        f"- Tone: {', '.join(guidelines.get('Tone', ['Professional and helpful']))}" if guidelines.get('Tone') else "- Tone: Professional and helpful",
        f"- Instructions: {', '.join(guidelines.get('Instructions', ['Provide accurate information']))}" if guidelines.get('Instructions') else "- Instructions: Provide accurate information",
        "",
        "SHOPLITE KNOWLEDGE BASE CONTEXT:",
        "=" * 80,
        context_text,
        "=" * 80,
        "",
        f"User Question: {query}",
        "",
        "IMPORTANT: Extract specific details from the context above.",
        "",
        "Response Format Required:",
        response_format,
        "",
        "Your Response (extract specific details from context):"
    ])

    return "\n".join(prompt_parts)



def generate_out_of_scope_response(query: str, question_type: str) -> Dict:


    if question_type == 'greeting':
        return {
            "answer": "Hello! I'm the ShopLite customer service assistant. I can help you with questions about ShopLite's features, policies, and services including account management, shopping cart, payments, shipping, returns, and seller services. How can I assist you with ShopLite today?",
            "sources": ["none"],
            "confidence": "High"
        }

    elif question_type == 'assistant_info':
        return {
            "answer": "I'm a customer service assistant specifically designed to help with ShopLite-related questions. I can provide information about account management, shopping cart features, payment methods, shipping, returns, seller services, and other ShopLite policies based on our official documentation. What would you like to know about ShopLite?",
            "sources": ["none"],
            "confidence": "High"
        }

    else:
        return {
            "answer": "I'm sorry, but I can only assist with questions related to ShopLite's services, features, and policies. Your question appears to be outside my area of expertise. I can help you with ShopLite account management, shopping cart features, payment methods, shipping and delivery, returns and refunds, seller services, or product search. What would you like to know about ShopLite?",
            "sources": ["none"],
            "confidence": "High"
        }



def generate_response(query, retrieved_docs, prompt_type="base-retrieval-prompt", max_new_tokens=250):
    """Generate response using YAML prompts - FIXED for speed and accuracy"""
    global tokenizer, model

    if not retrieved_docs:
        return {
            "answer": "I'm sorry, I don't have information about that based on the official ShopLite documentation. Could you please rephrase your question or ask about a specific ShopLite feature?",
            "sources": ["none"],
            "confidence": "Low"
        }

    prompt_text = build_prompt_from_yaml(query, retrieved_docs, prompt_type)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1800
    )

    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    input_ids = inputs["input_ids"]
    attention_mask = inputs.get("attention_mask")

    try:
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                #min_new_tokens=20,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,
                no_repeat_ngram_size=2,
                early_stopping=True,
                length_penalty=0.8
            )

        input_length = input_ids.shape[1]
        generated_ids = outputs[0][input_length:]
        generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    except Exception as e:
        print(f"Generation error: {e}")
        if retrieved_docs:
            doc_titles = [doc.get('title', 'Unknown') for doc in retrieved_docs]
            answer_content = retrieved_docs[0]['content'][:150]
            return {
                "answer": f"Based on the ShopLite documentation ({doc_titles[0]}): {answer_content}",
                "sources": doc_titles,
                "confidence": "Medium"
            }
        else:
            return {
                "answer": "I'm sorry, I don't have information about that based on the official ShopLite documentation.",
                "sources": ["none"],
                "confidence": "Low"
            }


    if not generated_text or len(generated_text.strip()) < 10:
        if retrieved_docs:
            doc = retrieved_docs[0]
            content = doc['content']

            # FIXED: Extract specific information based on query type
            query_lower = query.lower()
            if 'courier' in query_lower or 'delivery' in query_lower:
                # Extract courier names specifically
                courier_matches = re.findall(r'(aramex lebanon|tnt lebanon|fetchr lebanon)', content, re.IGNORECASE)
                if courier_matches:
                    courier_list = ", ".join(set(courier_matches))
                    answer = f"ShopLite deliveries are handled by {courier_list}."
                else:
                    # Look for broader courier info
                    delivery_sentences = re.findall(r'[^.]*(?:courier|delivery|shipping)[^.]*\.', content, re.IGNORECASE)
                    answer = " ".join(delivery_sentences[:2]) if delivery_sentences else content[:150]
            elif 'payment' in query_lower and 'refund' in query_lower:
                # Look for payment and refund information
                payment_sentences = re.findall(r'[^.]*(?:payment|refund)[^.]*\.', content, re.IGNORECASE)
                answer = " ".join(payment_sentences[:3]) if payment_sentences else content[:200]
            else:
                sentences = re.split(r'[.!?]+', content)
                clean_sentences = [s.strip() for s in sentences if len(s.strip()) > 15]
                answer = ". ".join(clean_sentences[:2]) + "." if clean_sentences else content[:150]

            return {
                "answer": answer,
                "sources": [doc['title']],
                "confidence": "Medium"
            }
        else:
            return {
                "answer": "I'm sorry, I don't have information about that based on the official ShopLite documentation.",
                "sources": ["none"],
                "confidence": "Low"
            }

    lines = [line.strip() for line in generated_text.split('\n') if line.strip()]
    answer = ""
    sources = []
    confidence = "Medium"

# getting sure it returned the required format
    for i, line in enumerate(lines):
        line_lower = line.lower()
        if line_lower.startswith('answer:'):
            answer_text = line[7:].strip()
            j = i + 1
            while j < len(lines) and not lines[j].lower().startswith(('sources:', 'source:', 'confidence:')):
                answer_text += " " + lines[j]
                j += 1
            answer = answer_text
        elif line_lower.startswith(('sources:', 'source:')):
            sources_text = line.split(':', 1)[1].strip() if ':' in line else ""
            if sources_text and sources_text != "none":
                sources = [s.strip().strip('"[]') for s in sources_text.split(',')]
        elif line_lower.startswith('confidence:'):
            conf_text = line.split(':', 1)[1].strip() if ':' in line else ""
            conf_text = conf_text.strip('"').lower()
            if conf_text in ['low', 'medium', 'high']:
                confidence = conf_text.capitalize()


    if answer:
        # Validate that key information exists in source documents
        retrieved_content = " ".join([doc['content'].lower() for doc in retrieved_docs])
        answer_numbers = re.findall(r'\d+[-–]\d+|\d+', answer.lower())

        validation_failed = False
        for num in answer_numbers:
            if num not in retrieved_content:
                print(f"Warning: Number '{num}' not found in source documents")
                validation_failed = True
                break

        if validation_failed:
            doc = retrieved_docs[0]
            sentences = re.split(r'[.!?]+', doc['content'])
            clean_sentences = [s.strip() for s in sentences if len(s.strip()) > 15]
            answer = ". ".join(clean_sentences[:2]) + "." if clean_sentences else doc['content'][:150]

    # Fallback parsing if YAML format wasn't followed
    if not answer and retrieved_docs:
        doc = retrieved_docs[0]
        sentences = re.split(r'[.!?]+', doc['content'])
        clean_sentences = [s.strip() for s in sentences if len(s.strip()) > 15]
        answer = ". ".join(clean_sentences[:2]) + "." if clean_sentences else doc['content'][:150]

    # Apply length limits
    if answer:
        sentences = re.split(r'[.!?]+', answer)
        clean_sentences = [s.strip() for s in sentences if len(s.strip()) > 5]
        if len(clean_sentences) > 4:
            answer = ". ".join(clean_sentences[:4]) + "."
        elif not answer.rstrip().endswith(('.', '!', '?')):
            if clean_sentences:
                answer = ". ".join(clean_sentences) + "."

    # Ensure we have sources
    if not sources:
        sources = [doc.get('title', 'Unknown') for doc in retrieved_docs] if retrieved_docs else ["none"]

    return {
        "answer": answer,
        "sources": sources,
        "confidence": confidence
    }



def generate_response_auto(query, retrieved_docs, embedder=None):


    is_related, confidence_score = is_shoplite_related(query)

    if not is_related:
        question_type = detect_question_type(query)
        return generate_out_of_scope_response(query, question_type)

    # If no documents were retrieved but query seems ShopLite-related
    if not retrieved_docs and is_related:
        return generate_response(query, [], prompt_type="refusal-prompt")

    # Determine prompt type based on query characteristics
    def is_vague(q):
        q_lower = q.lower()
        question_words = ["how", "what", "when", "where", "why", "can", "does", "is", "should",
                          "return", "refund", "payment", "shipping", "order", "account"]
        return len(q.split()) < 3 and not any(w in q_lower for w in question_words)

    if is_vague(query):
        prompt_type = "clarification-prompt"
    elif not retrieved_docs:
        prompt_type = "refusal-prompt"
    elif len(retrieved_docs) > 1:
        prompt_type = "multi-doc-retrieval-prompt"
    else:
        prompt_type = "base-retrieval-prompt"

    return generate_response(query, retrieved_docs, prompt_type=prompt_type)

In [ ]:
# test generated answers
setup_tokenizer_padding(tokenizer)
index, embedder = create_embeddings(EXPANDED_KB)


query = " Which couriers handle ShopLite deliveries?"
retrieved = retrieve_docs(query, EXPANDED_KB, index, embedder)
response = generate_response_auto(query, retrieved)
print(response)


query_out = "Which courier handle shoplite deleveries?"
retrieved_out = retrieve_docs(query_out, EXPANDED_KB, index, embedder)
response_out = generate_response_auto(query_out, retrieved_out)
print(response_out)


In [ ]:
index, embedder = create_embeddings(EXPANDED_KB)


app = Flask(__name__)
@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        query = data.get('query', '')
        if not query:
            return jsonify({"error": "Query not provided"}), 400

        # Retrieve relevant docs
        retrieved = retrieve_docs(query, EXPANDED_KB, index, embedder)

        # Generate response
        response = generate_response_auto(query, retrieved)

        return jsonify(response)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

#ping -> direct LLM without retrieval
@app.route('/ping', methods=['POST'])
def ping():
    try:
        data = request.get_json()
        query = data.get('query', '')
        if not query:
            return jsonify({"error": "Query not provided"}), 400

        inputs = tokenizer(query, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=150)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return jsonify({"answer": answer})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# health to check system status
@app.route('/health', methods=['GET'])
def health():
    try:
        status = {
            "status": "ok",
            "model_loaded": True,
            "knowledge_base_size": len(EXPANDED_KB)
        }
        return jsonify(status)
    except Exception as e:
        return jsonify({"error": str(e)}), 500


def start_flask():
    app.run(port=5000)

Thread(target=start_flask, daemon=True).start()

In [ ]:

ngrok_token = input("Enter your ngrok token: ")
ngrok.set_auth_token(ngrok_token)

public_url_obj = ngrok.connect(5000)
public_url = public_url_obj.public_url
print(f"Your app is live at: {public_url}")



In [ ]:



url = public_url


try:
    health_resp = requests.get(f"{url}/health", timeout=10)
    print("✅ /health response:")
    print("Status code:", health_resp.status_code)
    print("Body:", health_resp.json())
except Exception as e:
    print("❌ /health request failed:", e)
    print(traceback.format_exc())


try:
    ping_resp = requests.post(f"{url}/ping", json={"query": "Hello test"}, timeout=10)
    print("\n✅ /ping response:")
    print("Status code:", ping_resp.status_code)
    print("Answer:", ping_resp.json().get("answer", ""))
except Exception as e:
    print("❌ /ping request failed:", e)
    print(traceback.format_exc())

try:
    chat_resp = requests.post(f"{url}/chat", json={"query": "How do I create a ShopLite account?"}, timeout=10)
    chat_json = chat_resp.json()
    print("\n✅ /chat response:")
    print("Status code:", chat_resp.status_code)
    print("Answer:", chat_json.get("answer", ""))
    print("Sources:", chat_json.get("sources", []))
    print("Confidence:", chat_json.get("confidence", "Medium"))
except Exception as e:
    print("❌ /chat request failed:", e)
    print(traceback.format_exc())

In [ ]:


app = Flask(__name__)
run_with_ngrok(app)  # Uses your existing ngrok setup

@app.route('/generate', methods=['POST'])
def generate():
    prompt = request.json.get('prompt')
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=400)
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return jsonify({"text": response.strip()})

if __name__ == '__main__':
    app.run()